## Using Amazon SageMaker RL

Amazon SageMaker RL allows you to train your RL agents in cloud machines using docker containers. You do not have to worry about setting up your machines with the RL toolkits and deep learning frameworks. You can easily switch between many different machines setup for you, including powerful GPU machines that give a big speedup. You can also choose to use multiple machines in a cluster to further speedup training, often necessary for production level loads.

In [ ]:
import sagemaker
import boto3
import sys
import os
import glob
import re
import subprocess
from IPython.display import HTML
import time
from time import gmtime, strftime
from misc import get_execution_role, wait_for_s3_object, wait_for_training_job_to_complete
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework

### Setup S3 bucket

Set up the linkage and authentication to the S3 bucket that you want to use for checkpoint and the metadata. 

In [ ]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

### Define Variables 

We define variables such as the job prefix for the training jobs *and the image path for the container (only when this is BYOC).*

In [ ]:
# create a descriptive job name 
job_name_prefix = 'rl-battery'

### Configure where training happens

You can train your RL training jobs using the SageMaker notebook instance or local notebook instance. In both of these scenarios, you can run the following in either local or SageMaker modes. The local mode uses the SageMaker Python SDK to run your code in a local container before deploying to SageMaker. This can speed up iterative testing and debugging while using the same familiar Python SDK interface. You just need to set `local_mode = True`.

In [ ]:
# run in local_mode on this machine, or as a SageMaker TrainingJob?
local_mode = False

if local_mode:
    instance_type = 'local'
else:
    # If on SageMaker, pick the instance type
    instance_type = "ml.p3.2xlarge"
print(instance_type)

### Create an IAM role
Either get the execution role when running from a SageMaker notebook `role = sagemaker.get_execution_role()` or, when running locally, set it to an IAM role with `AmazonSageMakerFullAccess` and `CloudWatchFullAccess permissions`.

In [ ]:
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

### Install docker for `local` mode

In order to work in `local` mode, you need to have docker installed. When running from you local instance, please make sure that you have docker or docker-compose (for local CPU machines) and nvidia-docker (for local GPU machines) installed. Alternatively, when running from a SageMaker notebook instance, you can simply run the following script 

Note, you can only run a single local notebook at one time.

In [ ]:
# only run from SageMaker notebook instance
if local_mode:
    !/bin/bash ../src/common/setup.sh

### Define Metric
A list of dictionaries that defines the metric(s) used to evaluate the training jobs. Each dictionary contains two keys: ‘Name’ for the name of the metric, and ‘Regex’ for the regular expression used to extract the metric from the logs.

In [ ]:
metric_definitions = [{'Name': 'episode_reward_mean',
  'Regex': 'episode_reward_mean: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)'},
 {'Name': 'episode_reward_max',
  'Regex': 'episode_reward_max: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)'},
 {'Name': 'episode_len_mean',
  'Regex': 'episode_len_mean: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)'},
 {'Name': 'entropy',
  'Regex': 'entropy: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)'},
 {'Name': 'episode_reward_min',
  'Regex': 'episode_reward_min: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)'},
 {'Name': 'vf_loss',
  'Regex': 'vf_loss: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)'},
 {'Name': 'policy_loss',
  'Regex': 'policy_loss: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)'},                                            
]

# metric_definitions = RLEstimator.default_metric_definitions(RLToolkit.RAY)


In [ ]:
metric_definitions

### Define Estimator
This Estimator executes an RLEstimator script in a managed Reinforcement Learning (RL) execution environment within a SageMaker Training Job. The managed RL environment is an Amazon-built Docker container that executes functions defined in the supplied entry_point Python script.

In [ ]:
train_entry_point = "train_battery_sm.py"
train_job_max_duration_in_seconds = 60 * 60 * 48

hyperparameters =  {
"rl.training.config.env_config.MAX_STEPS_PER_EPISODE": 168,
"rl.training.config.env_config.LOCAL": True,
"rl.training.config.env_config.FILEPATH": "refdata/PRICE_AND_DEMAND_2020FULL_NSW1.csv",
"rl.training.config.use_pytorch": False,
"rl.training.stop.training_iteration": 5000,
"rl.training.run": "DQN",
}

estimator = RLEstimator(entry_point=train_entry_point,
                        source_dir="../src/share",
                        dependencies=["../src/common/sagemaker_rl", "../refdata"],
                        toolkit=RLToolkit.RAY,
                        toolkit_version='0.8.5',
                        framework=RLFramework.TENSORFLOW,
                        role=role,
                        instance_type=instance_type,
                        instance_count=1,
                        debugger_hook_config=False,
                        output_path=s3_output_path,
                        base_job_name=job_name_prefix,
                        metric_definitions=metric_definitions,
                        max_run=train_job_max_duration_in_seconds,
                        hyperparameters=hyperparameters,
                       )


In [ ]:
estimator.fit(wait=local_mode)
job_name=estimator._current_job_name
print("Job name: {}".format(job_name))

## Visualization

RL training can take a long time.  So while it's running there are a variety of ways we can track progress of the running training job.  Some intermediate output gets saved to S3 during training, so we'll set up to capture that.

In [ ]:
s3_url = "s3://{}/{}".format(s3_bucket,job_name)

intermediate_folder_key = "{}/output/intermediate/".format(job_name)
intermediate_url = "s3://{}/{}training/".format(s3_bucket, intermediate_folder_key)

print("S3 job path: {}".format(s3_url))
print("Intermediate folder path: {}".format(intermediate_url))

### Plot metrics for training job
We can see the reward metric of the training as it's running, using algorithm metrics that are recorded in CloudWatch metrics.  We can plot this to see the performance of the model over time.


In [ ]:
local_mode = False
job_name = "rl-battery-2021-05-10-09-49-21-387"

In [ ]:
%matplotlib inline
from sagemaker.analytics import TrainingJobAnalytics

if not local_mode:
    wait_for_training_job_to_complete(job_name) # Wait for the job to finish
    df = TrainingJobAnalytics(job_name, ['episode_reward_mean']).dataframe()
    df_min = TrainingJobAnalytics(job_name, ['episode_reward_min']).dataframe()
    df_max = TrainingJobAnalytics(job_name, ['episode_reward_max']).dataframe()
    df['rl_reward_mean'] = df['value']
    df['rl_reward_min'] = df_min['value']
    df['rl_reward_max'] = df_max['value']
    num_metrics = len(df)
    
    if num_metrics == 0:
        print("No algorithm metrics found in CloudWatch")

else:
    print("Can't plot metrics in local mode.")

In [ ]:
mean_reward = df['rl_reward_mean'].mean()
print(f"Average reward: {mean_reward}")
plt = df.plot(x='timestamp', y=['rl_reward_mean'], figsize=(20,5), fontsize=18, legend=True, style='-', color=['b','r','g'])
plt.set_ylabel('Mean reward per episode', fontsize=20)
plt.set_xlabel('Training time (s)', fontsize=20)
plt.axhline(y=mean_reward, color='r')
plt.grid()

## Save Metrics

In [ ]:
df.to_csv("metrics-rl-battery-2021-05-10-09-49-21-387.csv", index=False)